<a href="https://colab.research.google.com/github/nikhil-xyz/Text-classification/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/nikhil-xyz/Text-classification.git

Cloning into 'Text-classification'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 1), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), 18.41 MiB | 4.91 MiB/s, done.


In [2]:
import pandas as pd
data = pd.read_csv('/content/Text-classification/go_emotions_dataset.csv', on_bad_lines='skip')

In [3]:
data = data.head(10000)

In [4]:
data = pd.get_dummies(data, columns = ['example_very_unclear'])

In [5]:
data.drop(['example_very_unclear_False'], axis=1, inplace=True)

In [6]:
data = data.rename(columns = {'example_very_unclear_False':'example_very_unclear'})

In [7]:
data.shape

(10000, 31)

In [8]:
data.isnull().sum()

id                           0
text                         0
admiration                   0
amusement                    0
anger                        0
annoyance                    0
approval                     0
caring                       0
confusion                    0
curiosity                    0
desire                       0
disappointment               0
disapproval                  0
disgust                      0
embarrassment                0
excitement                   0
fear                         0
gratitude                    0
grief                        0
joy                          0
love                         0
nervousness                  0
optimism                     0
pride                        0
realization                  0
relief                       0
remorse                      0
sadness                      0
surprise                     0
neutral                      0
example_very_unclear_True    0
dtype: int64

In [9]:
import string
data['text'] = data['text'].apply(lambda word:[letters.lower() for letters in word if letters not in string.punctuation])
data['text'] = data['text'].apply(lambda word:''.join(word))

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [11]:
wnl = WordNetLemmatizer()

In [12]:
def preprocess(text):
  text = text.lower()
  text = text.split()
  text = [wnl.lemmatize(word) for word in text if not word in stopwords.words('english')]
  text = ' '.join(text)
  return text

In [13]:
data['text'] = data['text'].apply(preprocess)

In [14]:
data.iloc[1]['text']

'sexuality shouldn’t grouping category make different othet ppl imo fit definition grouping'

In [15]:
X = list(data['text'])
y = list(data[data.columns[2:]].values)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
y_test[7]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0])

In [18]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.8 MB/s eta 0:00:00


In [19]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [20]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [21]:
# import tensorflow as tf

# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_encodings),
#     y_train
# ))

# test_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(test_encodings),
#     y_test
# ))

In [22]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import torch
import numpy as np
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.FloatTensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, y_train)
test_dataset = Dataset(test_encodings, y_test)

In [24]:
sample = train_dataset[0]
sample.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [25]:
sample['labels']

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [26]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100
)


In [27]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", return_dict=True, num_labels=29)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

In [28]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5000
  Number of trainable parameters = 66975773


Step,Training Loss
100,0.608200
200,0.288100
300,0.167000
400,0.154800
500,0.154600
600,0.160700
700,0.160600
800,0.150600
900,0.149500
1000,0.144200


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuration saved in ./results/checkpoint-3

TrainOutput(global_step=5000, training_loss=0.128557910823822, metrics={'train_runtime': 358.2944, 'train_samples_per_second': 111.64, 'train_steps_per_second': 13.955, 'total_flos': 486637936080000.0, 'train_loss': 0.128557910823822, 'epoch': 5.0})

In [29]:
output = trainer.predict(test_dataset)[1]

***** Running Prediction *****
  Num examples = 2000
  Batch size = 16


In [30]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [32]:
accuracy = accuracy_score(y_test, output)
# precision = precision_score(y_test, output)
# recall = recall_score(y_test, output)
# f1 = f1_score(y_test, output)

In [33]:
print("accuracy: ", accuracy)
# print("precision: ", precision)
# print("Recall: ", recall)
# print("F1-score: ", f1)

accuracy:  1.0


In [37]:
trainer.save_model('/content/Text-classification/assignment')

Saving model checkpoint to /content/Text-classification/assignment
Configuration saved in /content/Text-classification/assignment/config.json
Model weights saved in /content/Text-classification/assignment/pytorch_model.bin


In [36]:
from google.colab import files
files.download('/content/assignment')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>